# Messages and Requests

## Install

In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

In [ ]:
server = sy.orchestra.launch(
    name="messages-requests-example-datasite-1-do", port=7021, reset=True, dev_mode=True
)

## Setup

For the purpose of this tutorial we are creating a very simple dataset, which is created and owner by the root client

In [ ]:
admin_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# third party
import numpy as np

dataset = sy.Dataset(
    name="my dataset",
    asset_list=[
        sy.Asset(name="my asset", data=np.array([1, 2, 3]), mock=np.array([1, 1, 1]))
    ],
)
admin_client.upload_dataset(dataset)

In [ ]:
admin_client.register(
    name="Jane Doe",
    email="jane@caltech.edu",
    password="abc123",
    password_verify="abc123",
    institution="Caltech",
    website="https://www.caltech.edu/",
)

In [ ]:
guest_client = server.client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
asset = guest_client.datasets[0].assets[0]

In [ ]:
@sy.syft_function_single_use()
def func():
    # stdlib
    import random

    data = list(range(100))
    return sum(data) / 100 + random.random()


@sy.syft_function_single_use(data=asset)
def mean(data):
    # stdlib
    import random

    return sum(data) / len(data) + +random.random()


@sy.syft_function_single_use(data=asset)
def reveal_data(data):
    return data

In [ ]:
# Create a project
new_project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[guest_client],
)
new_project

In [ ]:
new_project.create_code_request(func, guest_client)
new_project.create_code_request(mean, guest_client)
new_project.create_code_request(reveal_data, guest_client)

In [ ]:
project = new_project.send()
project

In [ ]:
project

## Messaging

### Check New Messages

In [ ]:
admin_client.notifications

### Send a Message

### Mark as Read or Unread

In [ ]:
message = admin_client.notifications[0]
message

In [ ]:
message.mark_read()

In [ ]:
admin_client.notifications

In [ ]:
message.mark_unread()

In [ ]:
admin_client.notifications

## Requests

### Approving

In [ ]:
admin_client.requests

In [ ]:
func_request = admin_client.requests[-3]
func_request

In [ ]:
func_request.approve()

In [ ]:
admin_client.requests

### Rejecting

In [ ]:
dangerous_request = admin_client.projects[0].requests[-1]
dangerous_request

In [ ]:
dangerous_request.code

In [ ]:
dangerous_request.deny(reason="Dangerous request, you just want to reveal the data!")

In [ ]:
admin_client.projects[0]